In [278]:
import numpy as np
import soundfile as sf

# Global Parameters
sampling_rate = 44100
baud_rate = 1200
f0 = 1200  # Base frequency
freq_shift = 1000  # Frequency shift for each bit value


# Derived parameter: Decimated sampling rate (to reduce computational load)
# decimated_sampling_rate = int(2 * (f0 + freq_shift) * 1.1)  # Nyquist rate with a 10% buffer
decimated_sampling_rate = int(2 * (f0 + freq_shift) * 20)  # Nyquist rate with a 10% buffer


def introduce_imperfections(audio_signal, NOISE=False, FREQ_OFFSET=False, PHASE_OFFSET=False, SILENCE=False):

    if SILENCE:
        # silence_duration = np.random.uniform(0.1, 1)  # Random silence duration between 0.1s to 1s
        silence_duration = 5 # Seconds
        silence_samples = int(silence_duration * decimated_sampling_rate)
        audio_signal = np.concatenate([np.zeros(silence_samples), audio_signal])

    # Additive Noise
    if NOISE:
        SNR_dB = 20  # Signal-to-Noise Ratio in dB
        SNR_linear = 10 ** (SNR_dB / 10)
        power_signal = np.mean(audio_signal**2)
        noise_variance = power_signal / SNR_linear
        noise = np.sqrt(noise_variance) * np.random.randn(len(audio_signal))
        audio_signal = audio_signal + noise

    if FREQ_OFFSET:
        freq_offset = 10  # in Hz
        t = np.arange(len(audio_signal)) / decimated_sampling_rate
        audio_signal = audio_signal * np.exp(1j * 2 * np.pi * freq_offset * t)

    if PHASE_OFFSET:
        phase_offset = np.pi / 4
        print("Adding phase offset of {} radians".format(phase_offset))
        audio_signal = audio_signal * np.exp(1j * phase_offset)

    # Return only the real part
    return np.real(audio_signal)


def modulator(bits, SAVE=True, filename="output.wav"):
    """
    Coherent FSK modulator that ensures phase continuity.
    """

    # Parameters
    samples_per_symbol = int(decimated_sampling_rate / baud_rate)
    
    t = np.arange(0, samples_per_symbol) / decimated_sampling_rate
    audio = []
    
    # Start with an initial phase of 0
    current_phase = 0
    
    for b in bits:
        f = f0 if b == 0 else f0 + freq_shift
        
        # Generate the sinusoid for this bit while maintaining phase continuity
        audio_chunk = np.sin(2.0 * np.pi * f * t + current_phase)
        
        # Update the phase for the next symbol
        current_phase += 2.0 * np.pi * f * t[-1]
        
        audio.extend(audio_chunk)
    
    audio = np.array(audio)
    
    # Save audio to file
    if SAVE:
        sf.write(filename, audio, decimated_sampling_rate)
        
    return audio


def demodulator(audio):
    """
    Demodulate the audio using coherent demodulation.
    """

    # Load the audio file
    # audio, sr = sf.read(filename)
    # print("samping rate", sr)
    audio = audio
    # TODO remove hardcoding of the sample rate (88000 ripped off from the audio file read)
    sr = 88000

    samples_per_symbol = int(sr / baud_rate)
    
    demodulated_bits = []
    
    # Start with an initial phase of 0
    reference_phase = 0
    
    for i in range(0, len(audio), samples_per_symbol):
        # Chunk is one symbol
        chunk = audio[i:i+samples_per_symbol]

        # Ensure chunk and reference signals are the same length
        if len(chunk) < samples_per_symbol:
            chunk = np.pad(chunk, (0, samples_per_symbol - len(chunk)))
        
        # Construct the reference signals for both frequencies with the current phase
        t = np.arange(0, samples_per_symbol) / decimated_sampling_rate
        ref_signal_0 = np.sin(2.0 * np.pi * f0 * t + reference_phase)
        ref_signal_1 = np.sin(2.0 * np.pi * (f0 + freq_shift) * t + reference_phase)
        
        # Compute the correlation with the reference signals
        correlation_0 = np.sum(chunk * ref_signal_0)
        correlation_1 = np.sum(chunk * ref_signal_1)
        
        # Determine the bit based on which correlation is higher
        bit = 0 if correlation_0 > correlation_1 else 1
        demodulated_bits.append(bit)
        
        # Update the reference phase for the next symbol
        dominant_freq = f0 if bit == 0 else f0 + freq_shift
        reference_phase += 2.0 * np.pi * dominant_freq * t[-1]

        if reference_phase > 2.0 * np.pi:
            reference_phase -= 2.0 * np.pi
    
    return demodulated_bits


# Generate random bits
whitenoise_bits = []
whitenoise_bits = np.random.randint(0, 2, 42)
start_stop_bits = [1,0,1,1,1,0,0,1]
preamble_bits = [0, 1, 0, 1, 0, 1, 0, 1, 0, 1]
data_bits = [1, 0, 1, 1, 0, 0, 1, 0, 0, 1]
# data_bits = np.random.randint(0, 2, 10)

# NOTE: Since this writes to a file, if preamble is not calculated before
# data modulation itself, we will write over the data modulation
preamble_audio = modulator(preamble_bits)

bits = np.concatenate([preamble_bits, data_bits])
audio = modulator(np.concatenate([whitenoise_bits, start_stop_bits, preamble_bits, data_bits]))


# Correlate the start of the received audio with the preamble to detect start
correlation = np.correlate(
    audio,
    preamble_audio,
    mode='valid'
)

start_idx = np.argmax(correlation)


demodulated_bits_refined = demodulator(audio[start_idx:])

# Print out BER percentage
# TODO: I'm getting as many bits that I'm passing in (500) as the demodulated bits because if there is silence at the beginning, the demodulated bits will be padded with 1s
print("BER:\t\t\t\t", np.sum(np.abs(np.array(bits) - np.array(
    demodulated_bits_refined[-len(bits):]
))) / len(bits) * 100, "%")
# print("CORRECT:\t\t\t", True if np.isin(bits, demodulated_bits_refined) else False)


audio_imperfect = introduce_imperfections(audio, NOISE=False, FREQ_OFFSET=False, PHASE_OFFSET=False, SILENCE=False)
sf.write("output.wav", audio_imperfect, decimated_sampling_rate)


print("")
print("Orignal bits:\t\t\t", bits)
# print("Demodulated bits:\t\t", np.concatenate([demodulated_bits_refined[-len(bits):]]))
print("Demodulated bits:\t\t", np.concatenate([demodulated_bits_refined]))


print("")
print("Whitenoise\t\t\t", whitenoise_bits)


BER:				 75.0 %

Orignal bits:			 [0 1 0 1 0 1 0 1 0 1 1 0 1 1 0 0 1 0 0 1]
Demodulated bits:		 [0 1 0 1 0 1 0 1 0 1 0 1 1 0 1 1 0 0 1 0 0 1 1]

Whitenoise			 [1 0 0 1 0 1 1 0 1 1 0 1 0 1 0 1 0 0 0 1 0 0 1 0 0 1 0 1 1 1 1 0 1 0 1 0 1
 1 1 1 1 1]
